In [0]:
from delta.tables import DeltaTable

In [0]:
bronze_path = "/Volumes/practice/practice/processed_files/bronze"
bronze_table = DeltaTable.forPath(spark, bronze_path)
df_bronze = bronze_table.toDF()

In [0]:
from pyspark.sql.functions import col

In [0]:
df_silver_new = df_bronze.filter(col("amount") > 0) \
    .dropDuplicates(["id"]) \
    .withColumnRenamed("sale_date", "date_of_sale")

In [0]:
silver_path = "/Volumes/practice/practice/processed_files/silver"

In [0]:
try:
    silver_table = DeltaTable.forPath(spark, silver_path)
    silver_table.alias("s").merge(
        df_silver_new.alias("d"),
        "s.id = d.id"
    ).whenNotMatchedInsertAll().execute()
except:
     df_silver_new.write.format("delta").mode("overwrite").save(silver_path)
